# Data Transformation Exploration (raw CSV → model-ready)

This notebook helps you validate the **data transformation pipeline** step-by-step before fitting Bayesian models.

## What you’ll get confidence in

- Input files load correctly
- Retailer-specific parsing rules are applied correctly (Circana vs Costco CRX)
- Expected columns exist after transformation
- Prices / sales / logs look sensible (no weird zeros, negative values, or extreme outliers)
- Retailer separation behaves as expected (`retailer_filter`)
- “Missing feature” masking works (e.g., Costco missing competitor / private label)
- You can export an auditable `prepared_data_from_notebook.csv`

## Prereqs

- Place your raw files in `data/`:
  - `data/bjs.csv`
  - `data/sams.csv`
  - `data/costco.csv` (optional)

- Install dependencies:

```bash
pip install -r requirements.txt
```

## Costco note

Costco CRX typically has a different schema than Circana. The pipeline handles this via **runtime YAML retailer contracts** (`data.retailer_data_contracts` in `config_template.yaml`). This notebook will attempt to load those contracts so Costco is parsed correctly.


In [ ]:
import os
import sys
from pathlib import Path

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Ensure repo root is on PYTHONPATH so imports work regardless of notebook CWD
REPO_ROOT = Path.cwd()
if (REPO_ROOT / "data_prep.py").exists() is False:
    # If we're running from notebooks/, go one level up
    REPO_ROOT = Path(__file__).resolve().parents[1] if "__file__" in globals() else Path.cwd().parents[0]

if str(REPO_ROOT) not in sys.path:
    sys.path.insert(0, str(REPO_ROOT))

from data_prep import ElasticityDataPrep, PrepConfig

pd.set_option("display.max_columns", 200)
pd.set_option("display.width", 180)

# Where outputs from this notebook will go
RESULTS_DIR = REPO_ROOT / "results"
RESULTS_DIR.mkdir(parents=True, exist_ok=True)

print("Notebook environment ready")
print("repo root:", REPO_ROOT.resolve())
print("results/ directory:", RESULTS_DIR.resolve())


In [ ]:
# ---- Configure paths (edit if your filenames differ) ----
BJS_PATH = "data/bjs.csv"
SAMS_PATH = "data/sams.csv"

# Auto-include Costco if the file exists (you can also hardcode a path string here)
_default_costco = "data/costco.csv"
COSTCO_PATH = _default_costco if (REPO_ROOT / _default_costco).exists() else None

print("BJS_PATH:", BJS_PATH)
print("SAMS_PATH:", SAMS_PATH)
print("COSTCO_PATH:", COSTCO_PATH)

# ---- Load runtime retailer contracts from config_template.yaml (recommended) ----
# These contracts are what allow Costco CRX (different schema) to be parsed correctly.
retailer_data_contracts = None
try:
    import yaml

    cfg_path = REPO_ROOT / "config_template.yaml"
    if cfg_path.exists():
        with open(cfg_path, "r") as f:
            cfg_yaml = yaml.safe_load(f) or {}
        retailer_data_contracts = (cfg_yaml.get("data") or {}).get("retailer_data_contracts")
        print("Loaded retailer_data_contracts from:", cfg_path)
    else:
        print("WARNING: config_template.yaml not found; proceeding without retailer_data_contracts")
except Exception as e:
    print("WARNING: could not load retailer_data_contracts (continuing without it). Error:", e)

# ---- Configure data preparation ----
# Use retailer_filter="All" to keep retailers separate (needed for hierarchical model).
# Use retailer_filter="Overall" to combine into one pooled dataset.

retailers_cfg = {
    "BJs": {"has_promo": True, "has_competitor": True},
    "Sams": {"has_promo": True, "has_competitor": True},
}

# Costco CRX typically has promo depth inputs but does not include Private Label rows for cross-price.
if COSTCO_PATH is not None:
    retailers_cfg["Costco"] = {"has_promo": True, "has_competitor": False}

cfg = PrepConfig(
    retailer_filter="All",
    include_seasonality=True,
    include_promotions=True,
    include_time_trend=True,

    # V2: enable base vs promo separation (default-on in the library)
    separate_base_promo=True,

    # Base price estimation guardrails (used if base sales cols are missing/undefined)
    base_price_proxy_window=8,
    base_price_imputed_warn_threshold=0.30,

    # Dependent variable rule: always model Volume Sales.
    # If a retailer file is missing `Volume Sales`, data prep computes:
    #   Volume Sales = Unit Sales × factor
    # Costco CRX commonly needs this.
    volume_sales_factor_by_retailer={"Costco": 2.0},

    retailers=retailers_cfg,
    retailer_data_contracts=retailer_data_contracts,
    verbose=True,
)

prep = ElasticityDataPrep(cfg)

df = prep.transform(
    bjs_path=BJS_PATH,
    sams_path=SAMS_PATH,
    costco_path=COSTCO_PATH,
)

df.shape

In [ ]:
# ---- Quick inspection ----
display(df.head(10))

# Columns
cols = sorted(df.columns.tolist())
print(f"Columns ({len(cols)}):")
print(cols)


In [ ]:
# ---- Retailer breakdown (if applicable) ----
if "Retailer" in df.columns:
    display(df["Retailer"].value_counts())
    display(df.groupby("Retailer").agg(
        n_rows=("Date", "size"),
        min_date=("Date", "min"),
        max_date=("Date", "max"),
    ).sort_values("n_rows", ascending=False))
else:
    print("No Retailer column (likely retailer_filter='Overall' or single-retailer filtering).")


In [ ]:
# ---- Numeric sanity checks ----
key_cols = [
    "Volume_Sales_SI","Volume_Sales_PL",
    "Price_SI","Price_PL",
    "Log_Volume_Sales_SI","Log_Price_SI","Log_Price_PL",
    "Promo_Intensity_SI",
    "Week_Number",
    "has_promo","has_competitor",
]
key_cols = [c for c in key_cols if c in df.columns]

display(df[key_cols].describe().T)

# Quick checks for suspicious values
if "Price_SI" in df.columns:
    print("Min Price_SI:", df["Price_SI"].min())
if "Volume_Sales_SI" in df.columns:
    print("Min Volume_Sales_SI:", df["Volume_Sales_SI"].min())

# Missingness report
missing_rate = df[key_cols].isna().mean().sort_values(ascending=False)
display(missing_rate.to_frame("missing_rate"))


In [ ]:
# ---- Availability flags sanity (Costco-style missing features) ----
if "Retailer" in df.columns and "has_promo" in df.columns:
    display(df.groupby("Retailer")[["has_promo","has_competitor"]].mean())

    if "Promo_Intensity_SI" in df.columns:
        display(df.groupby("Retailer")["Promo_Intensity_SI"].agg(["mean","min","max"]))
else:
    print("No availability flags found (expected if you did not pass cfg.retailers).")


In [ ]:
# ---- Basic plots (interim confidence) ----
df_plot = df.sort_values("Date")

plt.figure(figsize=(14, 4))
plt.plot(df_plot["Date"], df_plot["Volume_Sales_SI"], linewidth=1)
plt.title("Sparkling Ice Volume Sales over time")
plt.xlabel("Date")
plt.ylabel("Volume_Sales_SI")
plt.grid(alpha=0.3)
plt.show()

plt.figure(figsize=(6, 5))
plt.scatter(df_plot["Log_Price_SI"], df_plot["Log_Volume_Sales_SI"], s=12, alpha=0.4)
plt.title("Log Sales vs Log Own Price")
plt.xlabel("Log_Price_SI")
plt.ylabel("Log_Volume_Sales_SI")
plt.grid(alpha=0.3)
plt.show()


In [ ]:
# ---- Export prepared data for auditability ----
out_path = RESULTS_DIR / "prepared_data_from_notebook.csv"
df.to_csv(out_path, index=False)
print("Wrote:", out_path)


In [ ]:
# ---- V2 feature checks: base price + promo depth ----
v2_cols = ["Base_Price_SI", "Log_Base_Price_SI", "Promo_Depth_SI"]
v2_cols = [c for c in v2_cols if c in df.columns]

if v2_cols:
    display(df[v2_cols].describe().T)

    # Promo depth interpretation: values are relative price change vs base.
    #  - 0.00 means no discount (avg price == base price)
    #  - negative means discounted (e.g., -0.10 ≈ 10% off)
    plt.figure(figsize=(8, 4))
    plt.hist(df["Promo_Depth_SI"], bins=50, alpha=0.75, edgecolor="black")
    plt.title("Promo_Depth_SI distribution (negative = discount)")
    plt.xlabel("Promo_Depth_SI")
    plt.ylabel("Count")
    plt.grid(alpha=0.25)
    plt.show()
else:
    print("V2 columns not found. Check that PrepConfig(separate_base_promo=True) and input columns exist.")
